# Assignment 2a: Language Modelling via N-grams (5 Marks)

## Due: March 17, 2022

Welcome to the first part of Assignment 2. In this assignment you will implement N-gram language models as discussed in lecture 3 and 4 of the course. We will be working with the [WikiText-2 dataset](https://paperswithcode.com/dataset/wikitext-2) which consists of about 100 million tokens collected from Good and Featured articles from wikipedia.

In [ ]:
try:
    from google.colab import drive
    drive.mount('/content/gdrive')
    data_dir = "gdrive/MyDrive/PlakshaNLP/Assignment2a/data/wikitext-2"
except:
    data_dir = "/datadrive/t-kabir/work/repos/PlakshaNLP/source/Assignment2a/data/wikitext-2/"

Mounted at /content/gdrive


In [ ]:
# Install required libraries
!pip install numpy
!pip install pandas
!pip install nltk

In [ ]:
# We start by importing libraries that we will be making use of in the assignment.
import string
from collections import defaultdict
import numpy as np
import pandas as pd
import nltk
nltk.download("punkt")
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

We start by loading Wiki-Text dataset into memory. It consists of 3 text files one each for train, validation and test, consisting raw text data corresponding to different wikipedia articles.

In [ ]:
# Load train data
with open(f"{data_dir}/wiki.train.tokens") as f:
    wiki_train = f.read()

# Load validation data
with open(f"{data_dir}/wiki.valid.tokens") as f:
    wiki_valid = f.read()

# Load Test data
with open(f"{data_dir}/wiki.test.tokens") as f:
    wiki_test = f.read()
    
print(f"Datasets Loaded!")
print(f"Number of characters in train corpus: {len(wiki_train)}")
print(f"Number of characters in validation corpus: {len(wiki_valid)}")
print(f"Number of characters in test corpus: {len(wiki_test)}")

Datasets Loaded!
Number of characters in train corpus: 10780437
Number of characters in validation corpus: 1120192
Number of characters in test corpus: 1255018


In [ ]:
# Lets print first 100 characters of the train_corpus
wiki_train[:100]

' \n = Valkyria Chronicles III = \n \n Senjō no Valkyria 3 : <unk> Chronicles ( Japanese : 戦場のヴァルキュリア3 ,'

## Task 1: Fit n-gram probabilities (2 Marks)

N-gram language models are trained by estimating the joint and conditional probabilities of all the n-grams from the training corpus. Using these probabilities we can then sample next sequence of tokens given the context or compute the probability of a complete piece of text.

### Task 1.1: Unigram Probabilities (0.75 Mark)

We will start by estimating unigram probabilities from the corpus, which can be simply done by calculating the normalized frequency of each token in the corpus. Implement the `get_unigram_probs` function below that does that

In [ ]:
from collections import Counter
def get_unigram_probs(corpus):
    """
    Estimates the probability of each unigram in the text by calculating
    normalized frequency of each unigram (word).
    
    Inputs:
        - corpus (str): A python string consisting of a piece of text
    
    Returns:
        - unigram_probs (dict): A python dictionary with unigrams (words) as keys and 
                                their frequencies as values
                                
    Example Input: "I am Sam . Sam I am . I do not like green eggs and ham ."
    Expected Output: {'I': 0.17647058823529413,
             'am': 0.11764705882352941,
             'Sam': 0.11764705882352941,
             '.': 0.17647058823529413,
             'do': 0.058823529411764705,
             'not': 0.058823529411764705,
             'like': 0.058823529411764705,
             'green': 0.058823529411764705,
             'eggs': 0.058823529411764705,
             'and': 0.058823529411764705,
             'ham': 0.058823529411764705}
    
    Note: To break the text into tokens you can just use corpus.split() this time,
    Wiki corpus is already tokenized so just using split() will also work fine.
    **Do not use `nltk.tokenize.word_tokenize` here**

    Hint: `defaultdict` can often make life easier in such type of problems.
            https://docs.python.org/3/library/collections.html#collections.defaultdict
    """
    
    unigram_probs = {}
    
    # YOUR CODE HERE
    unigram_probs = defaultdict(lambda: defaultdict(lambda: 0))
    words=corpus.split()
    
    c=Counter(words)

    for i in words:
      unigram_probs[i]=c.get(i)/len(words)
    #raise NotImplementedError()
    #print(unigram_probs)    
    return unigram_probs

In [ ]:
def check_dicts_same(dict1, dict2):
    if not (isinstance(dict1, dict) or isinstance(dict1, defaultdict)):
        print("Your function output is not a dictionary!")
        return False
    if len(dict1) != len(dict2):
        return False
    
    for key in dict1:
        val1 = dict1[key]
        val2 = dict2[key]
        if isinstance(val1, float) and isinstance(val2, float):
            if not np.allclose(val1, val2, 1e-4):
                return False
        if val1 != val2:
            return False
    
    return True

print("Running Sample Test Case 1")
sample_corpus = 'I am Sam . Sam I am . I do not like green eggs and ham .'
exp_unigram_probs = {'I': 0.17647058823529413,
             'am': 0.11764705882352941,
             'Sam': 0.11764705882352941,
             '.': 0.17647058823529413,
             'do': 0.058823529411764705,
             'not': 0.058823529411764705,
             'like': 0.058823529411764705,
             'green': 0.058823529411764705,
             'eggs': 0.058823529411764705,
             'and': 0.058823529411764705,
             'ham': 0.058823529411764705}

output_unigram_probs = get_unigram_probs(sample_corpus)
print(f"Input Corpus: {sample_corpus}")
print(f"Unigram Probabilities: {output_unigram_probs}")
print(f"Expected Unigram Probabilities: {output_unigram_probs}")

assert check_dicts_same(exp_unigram_probs, output_unigram_probs)
print("Sample Test Case Passed")
print("****************************************\n")
    
print("Running Sample Test Case 2")
sample_corpus = 'john likes to watch movies mary likes movies too mary also likes to watch football games'
exp_unigram_probs = {'john': 0.0625,
                     'likes': 0.1875,
                     'to': 0.125,
                     'watch': 0.125,
                     'movies': 0.125,
                     'mary': 0.125,
                     'too': 0.0625,
                     'also': 0.0625,
                     'football': 0.0625,
                     'games': 0.0625}

output_unigram_probs = get_unigram_probs(sample_corpus)
print(f"Input Corpus: {sample_corpus}")
print(f"Unigram Probabilities: {output_unigram_probs}")
print(f"Expected Unigram Probabilities: {exp_unigram_probs}")

assert check_dicts_same(output_unigram_probs, exp_unigram_probs)
print("Sample Test Case Passed")
print("****************************************\n")


Running Sample Test Case 1
Input Corpus: I am Sam . Sam I am . I do not like green eggs and ham .
Unigram Probabilities: defaultdict(<function get_unigram_probs.<locals>.<lambda> at 0x7f6197c66f80>, {'I': 0.17647058823529413, 'am': 0.11764705882352941, 'Sam': 0.11764705882352941, '.': 0.17647058823529413, 'do': 0.058823529411764705, 'not': 0.058823529411764705, 'like': 0.058823529411764705, 'green': 0.058823529411764705, 'eggs': 0.058823529411764705, 'and': 0.058823529411764705, 'ham': 0.058823529411764705})
Expected Unigram Probabilities: defaultdict(<function get_unigram_probs.<locals>.<lambda> at 0x7f6197c66f80>, {'I': 0.17647058823529413, 'am': 0.11764705882352941, 'Sam': 0.11764705882352941, '.': 0.17647058823529413, 'do': 0.058823529411764705, 'not': 0.058823529411764705, 'like': 0.058823529411764705, 'green': 0.058823529411764705, 'eggs': 0.058823529411764705, 'and': 0.058823529411764705, 'ham': 0.058823529411764705})
Sample Test Case Passed
*************************************

Let's get the unigram probabilities from the entire training corpus now

In [ ]:
train_unigram_probs = get_unigram_probs(wiki_train)

In [ ]:
print(train_unigram_probs.get('Valkyria'))

2.6316943725602e-05


### Task 1.2: N-gram Probabilities (1.25 Marks)

Now let's estimate the probabilities for N-grams with N > 1. For N-grams instead of storing joint probability of each N-gram in the corpus i.e. p(w_n-N+1, ...,w_n-1, w_n) we store conditional probabilities p(w_n | w_n-N+1, ...,w_n-1), which can be calculated easily by using the following formula:

<img src="https://i.ibb.co/mtFcBfh/n-gram.jpg" alt="n-gram" border="0">

Here the term in numerator denotes the number of N-grams with the words w_n-N+1, ...,w_n-1, w_n and the term in denominator denotes the number of (N-1) grams with the words w_n-N+1, ...,w_n-1 .

Implement the `get_ngram_cond_probs` function below

In [ ]:
def get_ngram_cond_probs(corpus, N = 2):
    """
    Estimates the N-gram conditional probabilities from the corpus.
    
    Inputs:
        - corpus (str): A python string consisting of a piece of text
        - N (int) : Value of  N in N-gram
        
    Returns:
        - ngram_cond_probs (dict) : A dictionary with (N-1)-gram tuple as keys and values as dictionaries
                                    such that ngram_cond_probs[(w_n-N+1, ...,w_n-1)][w_n]
                                    stores the conditional probability p(w_n | w_n-N+1, ...,w_n-1)
                                    See the examples below for clarity
                                    
    Example Input: corpus = 'I am Sam . Sam I am .', N = 2
    Expected Output: 
    {
         ('I',): {'am': 1.0},
         ('am',): {'Sam': 0.5, '.': 0.5},
         ('Sam',): {'.': 0.5, 'I': 0.5},
         ('.',): {'Sam': 1.0}
 
     }
     Explanation: 'I' is always followed 'am' in the `corpus`, while 'am' is followed by 'Sam' once and '.' the other time
                     hence you see 0.5, 0.5 probabilities for these two tokens.
     
    Example Input: corpus = 'I am Sam . Sam I am .', N = 3
    Expected Output:
    {
         ('I', 'am'): {'Sam': 0.5, '.': 0.5},
         ('am', 'Sam'): {'.': 1.0},
         ('Sam', '.'): {'Sam': 1.0},
         ('.', 'Sam'): {'I': 1.0},
         ('Sam', 'I'): {'am': 1.0}
     }
 
    """

    
    ngram_cond_probs = {}
    
    # YOUR CODE HERE
    words=corpus.split()
    ngrams=[]
    #ngramsCountNext={}
    
    ngramsCountNext = defaultdict(lambda: defaultdict(lambda: 0))
    #print(words)
    for i in range(len(words)):
      if (i<len(words)-N+1):
        ngrams.append(words[i:i+N-1])
        if (tuple(words[i:i+N-1])) in ngramsCountNext:
          ngramsCountNext[tuple(words[i:i+N-1])][words[i+N-1]]+=1
          #print(tuple(words[i:i+N-1]))
        #ngramsCountNext[tuple(words[i:i+N-1])][words[i+N-1]]+=1
        else:
          ngramsCountNext[tuple(words[i:i+N-1])][words[i+N-1]]=1
        #ngramsCountNext[tuple(words[i:i+N-1])][words[i+N-1]]=1

    #print(ngrams)
    #print(ngramsCountNext)              

    #ngram_cond_probs = {}
    for seq in ngramsCountNext:
      total_count = sum(ngramsCountNext[seq].values())
      #print(seq,total_count)
      for next in ngramsCountNext[seq]:
        #print(next)
        ngramsCountNext[seq][next] = ngramsCountNext[seq][next]/total_count
    #raise NotImplementedError()'''
    print(ngramsCountNext.get('resulting',))
    return (dict(ngramsCountNext))

In [ ]:
def check_dicts_same_v2(dict1, dict2):
    if not (isinstance(dict1, dict) or isinstance(dict1, defaultdict)):
        print("Your function output is not a dictionary!")
        return False
    
    if len(dict1) != len(dict2):
        return False
        
    for key in dict1:
        val1 = dict1[key]
        val2 = dict2[key]
        if isinstance(val1, float) and isinstance(val2, float):
            if not np.allclose(val1, val2, 1e-4):
                return False
        if (isinstance(val1, dict) or isinstance(val1, defaultdict)) \
            and (isinstance(val2, dict) or isinstance(val2, defaultdict)):
            if not check_dicts_same_v2(val1, val2):
                return False
        if val1 != val2:
            return False
    
    return True

print("Running Sample Test Case 1")
sample_corpus = 'I am Sam . Sam I am .'
N = 2
exp_ngram_probs =     {
         ('I',): {'am': 1.0},
         ('am',): {'Sam': 0.5, '.': 0.5},
         ('Sam',): {'.': 0.5, 'I': 0.5},
         ('.',): {'Sam': 1.0}
     }

output_ngram_probs = get_ngram_cond_probs(sample_corpus, N = N)
print(f"Input Corpus: {sample_corpus}, N = {N}")
print(f"N-gram Probabilities: {output_ngram_probs}")
print(f"Expected N-gram Probabilities: {exp_ngram_probs}")
print(output_ngram_probs.get(('Sam',)))
assert check_dicts_same_v2(output_ngram_probs, exp_ngram_probs)
print("Sample Test Case Passed")
print("****************************************\n")

print("Running Sample Test Case 2")
sample_corpus = 'I am Sam . Sam I am .'
N = 3
exp_ngram_probs =         {
         ('I', 'am'): {'Sam': 0.5, '.': 0.5},
         ('am', 'Sam'): {'.': 1.0},
         ('Sam', '.'): {'Sam': 1.0},
         ('.', 'Sam'): {'I': 1.0},
         ('Sam', 'I'): {'am': 1.0}
     }

output_ngram_probs = get_ngram_cond_probs(sample_corpus, N = N)
print(f"Input Corpus: {sample_corpus}, N = {N}")
print(f"N-gram Probabilities: {output_ngram_probs}")
print(f"Expected N-gram Probabilities: {exp_ngram_probs}")

assert check_dicts_same_v2(output_ngram_probs, exp_ngram_probs)
print("Sample Test Case Passed")
print("****************************************\n")

print("Running Sample Test Case 3")
sample_corpus = 'I am Sam . Sam I am .'
N = 4
exp_ngram_probs =  {('I', 'am', 'Sam'): {'.': 1.0},
             ('am', 'Sam', '.'): {'Sam': 1.0},
             ('Sam', '.', 'Sam'): {'I': 1.0},
             ('.', 'Sam', 'I'): {'am': 1.0},
             ('Sam', 'I', 'am'): {'.': 1.0}}

output_ngram_probs = get_ngram_cond_probs(sample_corpus, N = N)
print(f"Input Corpus: {sample_corpus}, N = {N}")
print(f"N-gram Probabilities: {output_ngram_probs}")
print(f"Expected N-gram Probabilities: {exp_ngram_probs}")

assert check_dicts_same_v2(output_ngram_probs, exp_ngram_probs)
print("Sample Test Case Passed")
print("****************************************\n")


Running Sample Test Case 1
None
Input Corpus: I am Sam . Sam I am ., N = 2
N-gram Probabilities: {('I',): defaultdict(<function get_ngram_cond_probs.<locals>.<lambda>.<locals>.<lambda> at 0x7f6197c82680>, {'am': 1.0}), ('am',): defaultdict(<function get_ngram_cond_probs.<locals>.<lambda>.<locals>.<lambda> at 0x7f6197c829e0>, {'Sam': 0.5, '.': 0.5}), ('Sam',): defaultdict(<function get_ngram_cond_probs.<locals>.<lambda>.<locals>.<lambda> at 0x7f6197c82440>, {'.': 0.5, 'I': 0.5}), ('.',): defaultdict(<function get_ngram_cond_probs.<locals>.<lambda>.<locals>.<lambda> at 0x7f6197c82f80>, {'Sam': 1.0})}
Expected N-gram Probabilities: {('I',): {'am': 1.0}, ('am',): {'Sam': 0.5, '.': 0.5}, ('Sam',): {'.': 0.5, 'I': 0.5}, ('.',): {'Sam': 1.0}}
defaultdict(<function get_ngram_cond_probs.<locals>.<lambda>.<locals>.<lambda> at 0x7f6197c82440>, {'.': 0.5, 'I': 0.5})
Sample Test Case Passed
****************************************

Running Sample Test Case 2
None
Input Corpus: I am Sam . Sam I am .

We can now generate N-gram probability distributions for our dataset

In [ ]:
print("Generating Bigram Distribution")
bigram_prob_dist = get_ngram_cond_probs(wiki_train, N = 2)

print("Generating Trigram Distribution")
trigram_prob_dist = get_ngram_cond_probs(wiki_train, N = 3)

print("Generating 4-gram Distribution")
tetragram_prob_dist = get_ngram_cond_probs(wiki_train, N = 4)

Generating Bigram Distribution
None
Generating Trigram Distribution
None
Generating 4-gram Distribution
None


## Task 2: Evaluating the language models (2.5 Marks)

Now that we have generated the N-gram distributions for various Ns we can evaluate the language models both quantitatively using the perplexity metric as well as qualitatively by generating text using these models.

### Task 2.1: Calculating Perplexity (0.5 Marks)

As discussed in Lecture 4, perplexity of a probability distribution is defined as:

<img src="https://i.ibb.co/swXqFCz/Perplexity.jpg" alt="Perplexity" border="0">

For the purposes of this assignment we will only calculate perplexity of a unigram language model. Finding perplexity of N-gram models for N > 1 requires smoothing which is beyond the scope of this assignment, but interested students can read about it [here](https://web.stanford.edu/~jurafsky/slp3/3.pdf). For now, implement the `get_unigram_perplexity` function below

In [ ]:
import math
def get_unigram_perplexity(corpus, unigram_probs):
    """
    Calculates the perplexity of a unigram language model on a text corpus
    
    Inputs:
        - corpus (str): A python string consisting of a piece of text    
        - unigram_probs (dict): A python dictionary with unigrams (words) as keys and 
                                their probabilities as values
    Returns:
        - perplexity (float) : Perplexity of unigram model on corpus
    
    Note 1: You can assume that each word/unigram that appears in the corpus, we have it's probability in `unigram_probs`
    Note 2: Use log to the base 2 for calculating entropy
    """
    
    perplexity = None
    H=0
    p=0
    # YOUR CODE HERE
    for i in corpus.split():
      p=unigram_probs.get(i)
      H=H+(-1*(p*math.log(p,2)))
    #raise NotImplementedError()
    perplexity=math.pow(2,H)
    return perplexity

In [ ]:
# Perplexity for simple models like unigram can blow up easily for large corpora.
# And Hence we only evaluate on the first 100 tokens of each data split 
wiki_train_subsample = " ".join(wiki_train.split()[:100])
wiki_valid_subsample = " ".join(wiki_valid.split()[:100])
wiki_test_subsample = " ".join(wiki_test.split()[:100])
#print(wiki_train_subsample)

train_ppl = get_unigram_perplexity(wiki_train_subsample, train_unigram_probs)
expected_train_ppl = 66.87292185710841
print(f"Training Data Perplexity: {train_ppl}")
print(f"Expected Training Data Perplexity: {expected_train_ppl}")
assert np.allclose(train_ppl, expected_train_ppl, 1e-4)

print("**********************************\n")

valid_ppl = get_unigram_perplexity(wiki_valid_subsample, train_unigram_probs)
expected_valid_ppl = 73.3487215828768
print(f"Validation Data Perplexity: {valid_ppl}")
print(f"Expected Validation Data Perplexity: {expected_valid_ppl}")
assert np.allclose(valid_ppl, expected_valid_ppl, 1e-4)

print("**********************************\n")

test_ppl = get_unigram_perplexity(wiki_test_subsample, train_unigram_probs)
expected_test_ppl = 54.97601260063733
print(f"Test Data Perplexity: {test_ppl}")
print(f"Expected Test Data Perplexity: {expected_test_ppl}")
assert np.allclose(test_ppl, expected_test_ppl, 1e-4)

print("**********************************\n")


Training Data Perplexity: 66.87292185710841
Expected Training Data Perplexity: 66.87292185710841
**********************************

Validation Data Perplexity: 73.3487215828769
Expected Validation Data Perplexity: 73.3487215828768
**********************************

Test Data Perplexity: 54.976012600637404
Expected Test Data Perplexity: 54.97601260063733
**********************************



### Task 2.2: Generating texts using N-gram LMs (2.5 Marks)

Using the estimated N-gram probabilities we can use the N-gram language models to generate pieces of text. We start with N-1 tokens and then use them as context to predict the next token form our estimated distribution. This process is then done repeatedly till we reach the maximum specified length. Implement the `generate_text_unigram` and `generate_text_ngram` functions below

In [ ]:
import random
import numpy as np
def generate_text_unigram(unigram_probs, max_len = 10):
    """
    Generates a random piece of text by sampling from `unigram_probs`
    
    Inputs:
        - unigram_probs (dict): A dictionary containing probabilities of each unigram in the dataset.
        - max_len (int) : Maximum length of the sequence (in terms of number of words/tokens) to be generated
        
    Returns:
        - gen_text (str) : Text sampled from the unigram model
        
    Hint: np.random.choice might come in handy (pay special attention to what to supply as the value of its argument `p`)
    """
    
    gen_text = None
    words=list(unigram_probs.keys())
    #print(words)
    gen_text=np.random.choice(words,10,p=list(unigram_probs.values()))
    # YOUR CODE HERE
    #print(gen_text)

    
    


    #raise NotImplementedError()
    
    return " ".join(gen_text)

In [ ]:
np.random.seed(42)
gen_text = generate_text_unigram(train_unigram_probs, 10)
expected_text = 'The sustainable 2013 located of of , teeth became International'
print(f"Generated Text: {gen_text}")
print(f"Expected Text: {expected_text}")
assert gen_text == expected_text


Generated Text: The sustainable 2013 located of of , teeth became International
Expected Text: The sustainable 2013 located of of , teeth became International


In [ ]:
print(bigram_prob_dist.get(('resulting',)))

defaultdict(<function get_ngram_cond_probs.<locals>.<lambda>.<locals>.<lambda> at 0x7f619071ae60>, {'in': 0.562962962962963, 'savings': 0.007407407407407408, 'culture': 0.007407407407407408, 'from': 0.08888888888888889, 'film': 0.014814814814814815, 'loss': 0.007407407407407408, 'social': 0.007407407407407408, 'satire': 0.007407407407407408, 'oil': 0.007407407407407408, 'mixture': 0.007407407407407408, 'cricket': 0.007407407407407408, 'court': 0.007407407407407408, 'unit': 0.007407407407407408, 'sets': 0.007407407407407408, 'warfare': 0.007407407407407408, 'calculation': 0.007407407407407408, 'regime': 0.007407407407407408, 'highway': 0.007407407407407408, 'lawsuit': 0.007407407407407408, 'spin': 0.007407407407407408, 'if': 0.007407407407407408, '<unk>': 0.007407407407407408, 'Colorado': 0.007407407407407408, 'stresses': 0.007407407407407408, 'scuttling': 0.007407407407407408, 'magazine': 0.007407407407407408, 'album': 0.007407407407407408, 'fire': 0.007407407407407408, 'flood': 0.0074

In [ ]:
def generate_text_ngram(ngram_probs, seed_text, max_len = 10):
    """
    Generates a random piece of text by sampling from `ngram_probs`
    
    Inputs:
        - ngram_probs (dict): A dictionary containing containing conditional probabilities for each N-gram in the dataset.
        - seed_text (str): A string containing N-1 tokens to use for starting the generated sequence
        - max_len (int) : Maximum length of the sequence (in terms of number of words/tokens) to be generated
        
    Returns:
        - gen_text (str) : Text sampled from the ngram model
        
    Hint: np.random.choice might come in handy (pay special attention to what to supply as the value of its argument `p`)
    
    Examples:
        - Let's say we want to generate text from a bigram model (N = 2). The seed text will always have N-1 tokens i.e.
            in this case 1. For the case where seed_text is "The", you look up at all bigrams starting with "The" in `ngram_probs`.
            let's say the sampled word is "brown", so next you look at all the bigrams starting with "brown" and sample the
            next word and repeat the procedure.
        - For a trigram model and seed_text "The brown", you will similarly start by looking at the trigrams strating with
            "The brown", sample next word. let's say it is "fox", then you look for all the trigrams starting
            with "brown fox" and sample the next word and so on.
    """
    
    gen_text = []
    seed=seed_text.split()
    gen_text=seed
    #print(seed)
    l1=len(seed)
    while len(gen_text)<(max_len+l1):
      seed_text=gen_text[-l1:]
      words=ngram_probs.get(tuple(seed_text))
      #print(words)
      gen_text.append((np.random.choice(list(words.keys()),p=list(words.values()))))
      
      #print(gen_text)
      #print(seed_text)
    # YOUR CODE HERE

    #raise NotImplementedError()
    
    return " ".join(gen_text)

In [ ]:
np.random.seed(42)
gen_text = generate_text_ngram(bigram_prob_dist, "The", max_len= 20)
expected_text = "The resulting March 1944 . The song , piano at Victoria . Relay events of the United States , was an"
print(f"Generated Text: {gen_text}")
print(f"Expected Text: {expected_text}")
assert gen_text == expected_text

Generated Text: The resulting March 1944 . The song , piano at Victoria . Relay events of the United States , was an
Expected Text: The resulting March 1944 . The song , piano at Victoria . Relay events of the United States , was an


In [ ]:
np.random.seed(42)
gen_text = generate_text_ngram(trigram_prob_dist, "The resulting", max_len= 20)
expected_text = 'The resulting scuttling of the strongest aftershock since the <unk> Room . " Mosley \'s 1968 restoration . The bridge was written'
print(f"Generated Text: {gen_text}")
print(f"Expected Text: {expected_text}")
assert gen_text == expected_text

Generated Text: The resulting scuttling of the strongest aftershock since the <unk> Room . " Mosley 's 1968 restoration . The bridge was written
Expected Text: The resulting scuttling of the strongest aftershock since the <unk> Room . " Mosley 's 1968 restoration . The bridge was written


In [ ]:
np.random.seed(42)
gen_text = generate_text_ngram(tetragram_prob_dist, "= = =", max_len= 20)
expected_text = '= = = = Liu Kang was designed to produce . By the end of April . The months that receive the most'
print(f"Generated Text: {gen_text}")
print(f"Expected Text: {expected_text}")
assert gen_text == expected_text

Generated Text: = = = = Liu Kang was designed to produce . By the end of April . The months that receive the most
Expected Text: = = = = Liu Kang was designed to produce . By the end of April . The months that receive the most
